# ControlFlow Demo

Let's explore ControlFlow by building up from basic concepts to more complex workflows.

## Basic Tasks with cf.run()

The simplest way to use ControlFlow is with `cf.run()`. Let's start with the most basic example:

In [ ]:
import controlflow as cf

result = cf.run("Say hi!")


In [ ]:

print(f"\nThe agent said: {result}")

### Result Types

We can ensure we get exactly what we need using different result types:

In [ ]:
# Simple integer
number = cf.run(
    "Pick a number between 1 and 10",
    result_type=int
)

print(f"\nGot back: {number} (type: {type(number)})")

In [ ]:
from pydantic import BaseModel

# Structured data with Pydantic
class MovieReview(BaseModel):
    title: str
    rating: int  # 1-5 stars
    summary: str
    watch_again: bool

review = cf.run(
    "Review the latest movie you watched (I, Robot)",
    result_type=MovieReview
)


In [ ]:

print(f"\nMovie Review:")
print(f"Title: {review.title}")
print(f"Rating: {'⭐' * review.rating}")
print(f"Summary: {review.summary}")
print(f"Would watch again? {'Yes!' if review.watch_again else 'No'}")

### Classification

We can have agents classify content by providing valid options:

In [ ]:
# Classify news headlines by topic
headlines = [
    "Tech Giant Unveils Revolutionary AI Chip",
    "Scientists Discover New Species in Amazon",
    "Global Markets Rally on Economic Data",
]

CATEGORIES = ["Technology", "Science", "Business", "Politics", "Sports"]

classifications = []

for headline in headlines:
    
    category = cf.run(
        f"Classify this headline into exactly one category",
        context={"headline": headlines},
        result_type=list[CATEGORIES]
    )
    classifications.append((headline, category))


In [ ]:

print("\nClassified Headlines:")
for headline, category in classifications:
    print(f"📰 {headline}")
    print(f"   Category: {category}\n")

### Interactive Tasks

Agents can interact with users to gather information:

In [ ]:
from pydantic import BaseModel

class UserInfo(BaseModel):
    name: str
    age: int
    favorite_color: str

info = cf.run(
    "Have a friendly chat with the user to learn their name, age, and favorite color",
    interactive=True,
    result_type=UserInfo
)


In [ ]:

print(f"\nNice to meet you!")
print(f"Name: {info.name}")
print(f"Age: {info.age}")
print(f"Favorite color: {info.favorite_color}")

## Tools

Agents can use tools to extend their capabilities. Let's start with a simple random number generator:

In [ ]:
import random

def roll_die() -> int:
    """Roll a six-sided die and return the result."""
    return random.randint(1, 6)

result = cf.run(
    "Roll the die three times, and after each roll write a poem with that many lines",
    tools=[roll_die],
    result_type=list[int]
)


In [ ]:

print(f"\nRolled a {result}")

## Using Context

Context lets you provide additional information to help agents make decisions:

In [ ]:
def analyze_customer_feedback(feedback: str, product_type: str = None) -> str:
    """Analyze customer feedback with optional product context."""
    analysis = cf.run(
        "Analyze this customer feedback and determine if it's positive, negative, or neutral",
        context={
            "feedback": feedback,
            "product_type": product_type,
            "company_tone": "We aim for constructive, actionable insights"
        },
        result_type=["positive", "negative", "neutral"]
    )
    return analysis

# Example usage
feedback_1 = "This app crashes constantly! But at least the UI is pretty."
result_1 = analyze_customer_feedback(feedback_1, product_type="mobile app")

feedback_2 = "The new features are exactly what I needed!"
result_2 = analyze_customer_feedback(feedback_2, product_type="web service")

print("\nFeedback Analysis:")
print(f"1. \"{feedback_1}\"")
print(f"   Result: {result_1}")
print(f"\n2. \"{feedback_2}\"")
print(f"   Result: {result_2}")

## Tasks

While `cf.run()` is convenient, the `Task` class gives you more control:

In [ ]:
class Recipe(BaseModel):
    name: str
    ingredients: list[str]
    steps: list[str]
    prep_time: str
    difficulty: str

# Create the task
recipe_task = cf.Task(
    "Create a recipe for a quick and easy pasta dish",
    result_type=Recipe
)

# Run it
result = recipe_task.run()

print(f"\n🍝 {result.name}")
print(f"Difficulty: {result.difficulty}")
print(f"Prep time: {result.prep_time}")
print("\nIngredients:")
for item in result.ingredients:
    print(f"- {item}")
print("\nSteps:")
for i, step in enumerate(result.steps, 1):
    print(f"{i}. {step}")

In [ ]:
from pydantic import Field, BaseModel

class TravelPlan(BaseModel):
    destination: str
    duration_days: int = Field(ge=1, le=30)  # Between 1 and 30 days
    budget_per_day: float = Field(ge=50)     # At least $50/day
    activities: list[str] = Field(min_length=2, max_length=5)

travel_task = cf.Task(
    "Plan a fun weekend getaway",
    result_type=TravelPlan
)

plan = travel_task.run()

print(f"\n✈️ Travel Plan")
print(f"Destination: {plan.destination}")
print(f"Duration: {plan.duration_days} days")
print(f"Daily Budget: ${plan.budget_per_day:,.2f}")
print("\nActivities:")
for activity in plan.activities:
    print(f"- {activity}")

### Task State

Tasks are like contracts between you and the agent. Control is yielded to the agent for as long as the task remains incomplete.

In [ ]:
task = cf.Task(
    "Use the `weather` tool to get the weather in San Francisco",
)

task.run()

In [ ]:
print(task.status)
print(task.result)

## Flow and History

By default, each task runs independently. Let's see what happens when we try to reference a previous result:

In [ ]:
# Without Flow - no history
dice_roll = cf.run(
    "Roll a die and tell me what you got",
    tools=[roll_die],
    result_type=int
)

print(f"\nRolled: {dice_roll}")

# Try to reference the previous roll (this won't work!)
sum_result = cf.run(
    "Add 100 to the previous roll",
    result_type=int
)

print(f"\nResult (incorrect): {sum_result}")

Now let's use a Flow to maintain history between tasks:

In [ ]:
with cf.Flow() as flow:
    # First roll
    dice_roll = cf.run(
        "Roll a die and tell me what you got",
        tools=[roll_die],
        result_type=int
    )
    
    print(f"\nRolled: {dice_roll}")
    
    # Now the agent can remember the previous roll
    sum_result = cf.run(
        "Add 100 to the previous roll",
        result_type=int
    )
    
    print(f"\nResult (correct): {sum_result}")

## Memory

For persistence across different flows, we can use memory:

In [ ]:
# Create a memory for user preferences
preferences = cf.Memory(
    key="pizza_preferences",
    instructions="Store and recall information about pizza preferences"
)

# First interaction - learn preferences
with cf.Flow() as flow:
    cf.run(
        "Ask the user about their ideal pizza toppings",
        memories=[preferences],
        interactive=True
    )

print("\n(Let's pretend some time has passed...)\n")

# Second interaction - use those preferences
with cf.Flow() as flow:
    suggestion = cf.run(
        "Suggest a pizza order based on what you remember about their preferences",
        memories=[preferences],
        result_type=str
    )
    
    print(f"\nSuggested order: {suggestion}")

## Instructions

We can provide temporary guidance to agents:

In [ ]:
# Normal response
result = cf.run("In two sentences, explain quantum physics.")

print(result)



In [ ]:
# Now with pirate instructions
with cf.instructions("Talk like a pirate"):
    result = cf.run("In two sentences, explain quantum physics.")
    
    print(result)

In [ ]:

# And with poet instructions
with cf.instructions("ELI5"):
    result = cf.run("In two sentences, explain quantum physics.")
    
    print(result)

## Multi-Agent Collaboration

Different agents can work together, each bringing their own perspective:

In [ ]:
# Create agents with different perspectives
optimist = cf.Agent(
    name="Optimist",
    instructions="You always look for positive aspects and opportunities in any situation."
)

critic = cf.Agent(
    name="Critic",
    instructions="You carefully identify potential issues and areas for improvement."
)

mediator = cf.Agent(
    name="Mediator",
    instructions="You balance different perspectives and find practical solutions."
)

with cf.Flow() as flow:
    # First get different perspectives
    cf.run(
        "Evaluate this new product idea: A smart coffee mug that maintains your drink at the perfect temperature and tracks your caffeine intake",
        agents=[optimist, critic]
    )
    
    # Then have the mediator summarize
    summary = cf.run(
        "Based on the discussion, provide a balanced assessment and recommendation",
        agents=[mediator],
        result_type=str
    )
    
    print(f"\nFinal Assessment: {summary}")

## Let's Play a Game!

Let's combine everything we've learned into a game of rock, paper, scissors:

In [ ]:
# Create a memory for game stats
game_memory = cf.Memory(
    key="rps_stats",
    instructions="Track rock, paper, scissors game statistics"
)

with cf.Flow() as game:
    # Get the user's choice first
    user_choice = cf.run(
        "Ask the user to choose rock, paper, or scissors",
        interactive=True,
        result_type=["rock", "paper", "scissors"]
    )
    
    print("\n---\n")
    
    # Now get AI's choice (in a new flow to avoid seeing user's choice)
    with cf.Flow():
        ai_choice = cf.run(
            "Choose rock, paper, or scissors",
            result_type=["rock", "paper", "scissors"]
        )
    
    # Add some excitement with instructions
    with cf.instructions("Be an enthusiastic game show host"):
        cf.run(
            "Announce both choices and declare the winner!",
            memories=[game_memory],
            context={"ai_choice": ai_choice, "user_choice": user_choice}
        )